In [1]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

from datetime import datetime
import numpy as np
from copy import deepcopy
from preprocessing import generate_triple, generate_trn, row_normlize, read_file
import model
from incremental_learning import split_tripple,cal_l0,calc_l1,calc_l2,gene_incre_matrix,incre_cgl_rank_new

In [2]:
data_file = 'data/all_ruc_word_info_new_with_essay.csv'
link_file = 'data/all_ruc_new_link_with_essay.csv'
concept_file = 'data/ruc_all_concepts_new_with_essay.csv'
file_type = 'sparse_row_col_val'
incre_course_num = 176
incre_concept_num = 137
undirect=False

In [3]:
X, links, concept = read_file(data_file, link_file, concept_file=concept_file, file_type=file_type)

In [15]:
X = X.todense()
X = row_normlize(X)
n_course, n_concept = X.shape[0], X.shape[1]
trn = generate_trn(links, n_course, undirect=undirect)
tripple = generate_triple(trn)
split_tripple_list = split_tripple(tripple, range(X.shape[0]-incre_course_num,X.shape[0]))

In [48]:
T = split_tripple_list[0]
A, F, st = model.cgl_rank(X[:-incre_course_num, :-incre_concept_num], T, lamb=0.01,
                      eta=1, tolerence=10, silence=False)



Iteration: 10, eta: 0.157779 B's 'f-norm' decreases: 1376.477362, old_loss: 33697.824211, current_loss: 32321.346849
B change: max=44.61934559388539, min=-21.03476211160803
regulazition part=392.175018333372, loss_part=31929.171830278792
Iteration: 20, eta: 0.142396 B's 'f-norm' decreases: 205.997013, old_loss: 24463.849063, current_loss: 24257.852050
B change: max=40.53648507920489, min=-21.8822499593565
regulazition part=317.61849555881986, loss_part=23940.233554260427
Iteration: 30, eta: 0.142396 B's 'f-norm' decreases: 193.340217, old_loss: 22701.164021, current_loss: 22507.823803
B change: max=38.87024609247464, min=-24.66460833359691
regulazition part=307.59866015724845, loss_part=22200.225143075906
Iteration: 40, eta: 0.128512 B's 'f-norm' decreases: 88.843325, old_loss: 20986.180381, current_loss: 20897.337056
B change: max=37.16819652422308, min=-28.264594774598027
regulazition part=309.34630684525825, loss_part=20587.99074906928
Iteration: 50, eta: 0.128512 B's 'f-norm' decre

Stop, iteration: 353, eta: 0.128512 B's 'f-norm' decreases: 9.971586, old_loss: 7498.174603, current_loss: 7488.203017
B change: max=255.6383000986713, min=-222.72018528115134
regulazition part=1474.6938092278642, loss_part=6013.509207331116


In [49]:
A = gene_incre_matrix(A, incre_concept_num)
print('\n\n\n')

In [51]:
A.shape

(2254, 2254)

In [44]:
def incre_cgl_rank_new(X, st_idx, tripple, split_tripple_list, A0, lamb, eta, tolerrence=0.00001, silence=False, 
                   update_A1=False):
    '''
    增量学习版本，可以支持一门课多个词的输入。
    '''
    # TODO: 这里需要改
    # X1~4分别表示分块矩阵的四块，按行编号。
    st = datetime.now()
    A = A0.copy()
    T,T1,T2,T3,T4,T5,T6,T7 = split_tripple_list
    row_st_idx, col_st_idx = st_idx
    X1 = X[:row_st_idx, :col_st_idx]
    X2 = X[:row_st_idx, col_st_idx:]
    X3 = X[row_st_idx:, :col_st_idx]
    X4 = X[row_st_idx:, col_st_idx:]
    incre_course_num = X.shape[0] - row_st_idx
    F = np.matmul(np.matmul(X, A), X.transpose())

    # loss function: max( (1-F_ij+F_ik),0 ) square.
    def loss_func(x): return max(
        (1 - F[x[0], x[1]] + F[x[0], x[2]]), 0)**2
    
    round_A2 = 0
    eta1 = eta
    old_loss = np.inf
    while True:
        F = np.matmul(np.matmul(X, A), X.transpose())
        p = A.shape[0] - col_st_idx
        # l1,l2的形状跟待更新的A2,A3一样
        l1 = calc_l1(A, X, st_idx, F, split_tripple_list)
        l2 = calc_l2(A, X, st_idx, F, split_tripple_list)
        
        if update_A1:
            l0 = cal_l0(A, X, st_idx, F, T, T1, T2, T3)
        # A1 after this round update.
        while True:
            F_old = F.copy()
            A_old = A.copy()
            A[:col_st_idx, col_st_idx:] -= eta1 * (lamb * A[:col_st_idx, col_st_idx:] + l1)
            A[col_st_idx:, :col_st_idx] -= eta1 * (lamb * A[col_st_idx:, :col_st_idx] + l2)
            if update_A1:
                A[:col_st_idx, :col_st_idx] -= eta1 * (lamb * A[:col_st_idx, :col_st_idx] + l0)
            F = np.matmul(np.matmul(X, A), X.transpose())
            
            total1 = 0
            correct1 = 0
            for i, j, k in T2:
                if F[i,j] - F[i,k] > 0:
                    correct1 += 1
                total1 += 1
            
            total2 = 0
            correct2 = 0
            for i,j,k in T3:
                if F[i,j] - F[i,k] > 0:
                    correct2 += 1
                total2 += 1
                    
            loss_part = sum(list(map(loss_func, T1))) + sum(list(map(loss_func, T2))) + sum(list(map(loss_func, T3))) + sum(list(map(loss_func, T4))) + sum(list(map(loss_func, T5)))+ sum(list(map(loss_func, T6))) + sum(list(map(loss_func, T7)))
            if update_A1:
                loss_part += sum(list(map(loss_func, T))) 
            reg_part = lamb/2 * np.sqrt((A[:col_st_idx, col_st_idx:]**2).sum())
            cur_loss = loss_part + reg_part
            unit_loss_change = (old_loss - cur_loss) / A[:col_st_idx, col_st_idx:].size
            loss_change = (old_loss - cur_loss)
        
            if unit_loss_change < 0:
                print('loss_part={}, reg_part={}'.format(loss_part, reg_part))
                eta1 *= 0.95
                F = F_old.copy()
                A = A_old.copy()
            else:
                break
            
        if not silence:
            if round_A2 % 1 == 0:
                print('round: {}, eta: {}, loss decrease: {}'.format(round_A2, eta1, loss_change))
                print('loss_part={}, reg_part={}'.format(loss_part, reg_part))
                print('loss_part in tripple:{}'.format(sum(list(map(loss_func, tripple)))))
        round_A2 += 1
        if loss_change < tolerrence:
            break
        old_loss = cur_loss
    print('train cost: {} step, final loss: {}, loss_change: {}'.format(round_A2, cur_loss, loss_change))
    F = np.matmul(np.matmul(X, A), X.transpose())
    ed = datetime.now()
    return A, F, (ed-st).total_seconds()

In [54]:
A1, F1, st = incre_cgl_rank_new(X, (n_course-incre_course_num, n_concept-incre_concept_num), tripple, split_tripple_list, A, eta=0.1, lamb=0.01,
                        tolerrence=1, update_A1=False)

round: 0, eta: 0.1, loss decrease: inf
loss_part=49608.6105660805, reg_part=0.06685968069834296
loss_part in tripple:55679.170401555704
round: 1, eta: 0.1, loss decrease: 1530.4603883556774
loss_part=48078.10511543277, reg_part=0.11192197274544911
loss_part in tripple:54231.9867164443
round: 2, eta: 0.1, loss decrease: 1017.5011915185969
loss_part=47060.564398134215, reg_part=0.15144775271117583
loss_part in tripple:53311.50891391373
round: 3, eta: 0.1, loss decrease: 722.1509767062525
loss_part=46338.378075285626, reg_part=0.18679389504335148
loss_part in tripple:52695.52664323622
round: 4, eta: 0.1, loss decrease: 533.6131875381107
loss_part=45804.73264075621, reg_part=0.21904088635102986
loss_part in tripple:52274.78804831724
round: 5, eta: 0.1, loss decrease: 403.69045336776617
loss_part=45401.01235314671, reg_part=0.2488751280859767
loss_part in tripple:51989.0298653127
round: 6, eta: 0.1, loss decrease: 309.7086606367375
loss_part=45091.275823309, reg_part=0.2767443290586562
loss

In [56]:
np.savetxt('result/ruc_A_incre_with_essay_direct_noupdate_A1.txt', A1)
np.savetxt('result/ruc_F_incre_with_essay_direct_noupdate_F1.txt', F1)

In [55]:
A1

array([[ 8.78348124e-05, -4.67662476e-06, -1.70331981e-04, ...,
        -5.79974912e-04,  0.00000000e+00, -8.42667718e-05],
       [ 1.28689770e-04,  1.65182848e-04,  6.91196805e-04, ...,
        -4.26344937e-04,  0.00000000e+00, -1.24315008e-04],
       [ 3.10425643e-04,  5.58342751e-04,  3.37912826e-03, ...,
        -7.42219080e-04,  0.00000000e+00, -1.67519058e-04],
       ...,
       [-4.35671316e-04, -3.05273623e-04, -1.62924047e-03, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.12214753e-04, -1.13695172e-04, -5.57439166e-04, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])